In [ ]:
!pip install transformers datasets torch peft accelerate

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model


device = "cuda" if torch.cuda.is_available() else "cpu"

# Model and tokenizer
model_name = "facebook/opt-2.7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to(device)


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Apply LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Load dataset
dataset = load_dataset("json", data_files="training_data.jsonl", split="train")

# Preprocess function: Map prompt to query
def preprocess_function(examples):
    # Inputs: Natural language prompts with prefix
    inputs = [f"### Prompt: {p}" for p in examples["prompt"]]
    # Outputs: Structured queries with prefix
    outputs = [f"### Query: {q}" for q in examples["query"]]

    tokenized_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Tokenize outputs as labels
    tokenized_outputs = tokenizer(
        outputs,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Set labels for training
    tokenized_inputs["labels"] = tokenized_outputs["input_ids"]
    return tokenized_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["prompt", "query"]
)

# Split dataset (90% train, 10% eval)
train_size = int(0.9 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

# Training arguments
training_args = TrainingArguments(
    output_dir="./opt-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    max_grad_norm=0.3,
    report_to="none",
    fp16=True if device == "cuda" else False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train
print("Starting fine-tuning...")
trainer.train()

# Save model and tokenizer
model.save_pretrained("./opt-finetuned-adapters")
tokenizer.save_pretrained("./opt-finetuned-adapters")
print("Fine-tuning complete!")

# Download results
!zip -r opt_finetuned.zip ./opt-finetuned-adapters
from google.colab import files
files.download("opt_finetuned.zip")